In [1]:
! pip install autogen

In [12]:
import re

def extract_code_blocks_with_type(markdown_text):
    """
    Extract code blocks and their types from a Markdown string.

    Parameters:
        markdown_text (str): The Markdown content as a string.

    Returns:
        list: A list of tuples, each containing (code_type, code_content).
              If no type is specified, code_type will be an empty string.
    """
    # Regex pattern to match code blocks with or without a code type
    pattern = r"```(\w+)?\n(.*?)```"

    # Use re.DOTALL to capture code content spanning multiple lines
    matches = re.findall(pattern, markdown_text, re.DOTALL)

    # Normalize the results
    return [(code_type if code_type else "", code_content.strip()) for code_type, code_content in matches]

In [69]:
import os
from autogen import ConversableAgent


class DataExtractionAgent:
    data_extraction_system_message = "you are a helpful data processing agent, help to extract useful information to unstrcuture data"

    payload_extraction_system_message = """
    the input data is a debezium CDC data payload, our target is to extract the payload in after or payload:after into a new stream
    the source stream has just one string field with name raw
    
    here is are sample queries to extrac the after payload based on different types of debezium payload
    case1. when the after payload is in root layer
    select raw:after from source_stream_name where _tp_time > earliest_ts()
    case2. when the after payload is in field of payload
    select raw:payload:after from source_stream_name where _tp_time > earliest_ts()

    return which extract query should be used in markdown code with sql
    
    """

    target_schema_inference_system_message = """please generate DDL based on debezium payload
    case1. when the after payload is in root layer, using json object in the after field as input
    case2. when the after payload is in field of payload, 
     using the json string in the after field and only in the after field of payload as input
     No other fields should be considered, such as source, or schema etc
    
    here are the rules to follow
    * the DDL grammar follows ClickHouse style
    * the Table keyword MUST be replaced with Stream
    * all datatypes MUST be in lowercase, such uint32
    * all keywords MUST be in lowercase, such as nullable
    * all field names MUST keep same as in the json
    * composite types such as array, tuple, map cannot be nullable 
    * should use composite types like array, map or tuple to represent complex structure in the json
    * output should be put into markdown of sql
    * bool type is supported
    * available composite types are
        * array
        * tuple
        * map
    * for composite type, using tuple over map, as tulpe is more generic
    
    here is a sample of output DDL:
    ```sql
    CREATE STREAM target_stream
    (
      `cid` string,
      `gas_percent` float64,
      `in_use` bool,
      `latitude` float64,
      `longitude` float64,
      `locked` bool,
      `speed_kmh` float64,
      `time` string,
      `total_km` float64
    )
    ```
    """

    mv_extraction_system_message = """please create a materialized view to extraction information from source stream into target stream
    the source stream has just one string field with name raw
    here are the rules to following
    * the grammar follows ClickHouse style
    * all function name follows snake case, such as json_extract_array
    * all keywords MUST be in lowercase, such as nullable
    * using tuple for hierarchy case which is generic
    
    
    here is the grammar of materialized view
    CREATE MATERIALIZED VIEW [IF NOT EXISTS] <view_name>
    INTO <target_stream> AS <SELECT ...>

    NOTE, to extrat json with hierarchy, 
    this one is WRONG : json_extract_uint(raw, 'after.customer_id') AS customer_id
    extract target field does not support hierarchy
    SHOULD BE : json_extract_uint(raw:after, 'customer_id') AS customer_id,

    this one is WRONG : tuple_cast(json_extract_string(raw:payload:after, '_id.$oid')) AS _id,
    SHOULD BE : tuple_cast(json_extract_string(raw:payload:after:_id, '$oid')) AS _id,

    to construct or convert tuple type , call tuple_cast, for example:
    tuple_cast(a, b) AS tuple_field,
    there is no tuple() function, NEVER call tuple() function

    In case the payload contains complex composition and hierarchy, you should provide the conversion layer by layer, do not miss any middle layer
    here is a sample that one of the target field is a map, using array_map function to help
    array_map(
        x -> (
            (json_extract_string(x, 'date'), json_extract_float(x, 'amount'))
        ),
        json_extract_array(after:raw_data, 'transaction_history')
    ) as transaction_history

    please only use following available json extraction functions if required:
    * json_extract_int
    * json_extract_uint
    * json_extract_float
    * json_extract_bool
    * json_extract_string
    * json_extract_array

    """
    
    def __init__(self):
        self._llm_config = {
            "config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}], "temperature": 0
        }
        

        self.data_extraction_agent = ConversableAgent(
            "data_extraction_agent",
            system_message=self.data_extraction_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.payload_extraction_agent = ConversableAgent(
            "payload_extraction_agent",
            system_message=self.payload_extraction_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.target_schema_inference_agent = ConversableAgent(
            "target_schema_inference_agent",
            system_message=self.target_schema_inference_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

        self.mv_extraction_agent = ConversableAgent(
            "mv_extraction_agent",
            system_message=self.mv_extraction_system_message,
            llm_config=self._llm_config,
            code_execution_config=False,
            max_consecutive_auto_reply=1,
            human_input_mode="NEVER",
        )

    def pipeline(self, data, source_stream_name, target_stream_name):
        message = f'based on input data : {data} and source stream name {source_stream_name}'
        self.data_extraction_agent.initiate_chats(
            [
                {
                    "recipient": self.payload_extraction_agent,
                    "message": message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.target_schema_inference_agent,
                    "message": f'based on input data : {data} and target stream name {target_stream_name}',
                    "max_turns": 1,
                    "summary_method": "last_msg",
                },
                {
                    "recipient": self.mv_extraction_agent,
                    "message": f'please create materialized view to extrat information from source stream to target stream',
                    "max_turns": 1,
                    "summary_method": "last_msg",
                }
            ]
        )
        
        return self.payload_extraction_agent.last_message()['content'], self.target_schema_inference_agent.last_message()['content'], self.mv_extraction_agent.last_message()['content']

In [70]:
data = """{
	"before": null,
	"after": {
		"customer_id": 9,
		"name": "Johnathan Rodriguez",
		"email": "thomasramirez@example.org",
		"phone": "001-845-290-8721x77863",
		"address": "743 Cervantes Causeway Apt. 762\nPort Lauren, NY 12698"
	},
	"source": {
		"version": "3.0.6.Final",
		"connector": "postgresql",
		"name": "postgres",
		"ts_ms": 1740183762305,
		"snapshot": "false",
		"db": "lumi_credit",
		"sequence": "[\"27606456\",\"27606456\"]",
		"ts_us": 1740183762305810,
		"ts_ns": 1740183762305810000,
		"schema": "public",
		"table": "customers",
		"txId": 763,
		"lsn": 27606456,
		"xmin": null
	},
	"transaction": null,
	"op": "c",
	"ts_ms": 1740183762818,
	"ts_us": 1740183762818345,
	"ts_ns": 1740183762818345200
}"""

source_stream_name = "kafka_cdc_postgres_customers"
target_stream_name = "customers"

agent = DataExtractionAgent()

agent1_output, agent2_output, agent3_output = agent.pipeline(data, source_stream_name, target_stream_name)

code = extract_code_blocks_with_type(agent1_output)
print(f"code type {code[0][0]}, extraction sql : {code[0][1]}")

print(agent2_output)

print(agent3_output)


********************************************************************************
Starting a new chat....

********************************************************************************
data_extraction_agent (to payload_extraction_agent):

based on input data : {
	"before": null,
	"after": {
		"customer_id": 9,
		"name": "Johnathan Rodriguez",
		"email": "thomasramirez@example.org",
		"phone": "001-845-290-8721x77863",
		"address": "743 Cervantes Causeway Apt. 762
Port Lauren, NY 12698"
	},
	"source": {
		"version": "3.0.6.Final",
		"connector": "postgresql",
		"name": "postgres",
		"ts_ms": 1740183762305,
		"snapshot": "false",
		"db": "lumi_credit",
		"sequence": "["27606456","27606456"]",
		"ts_us": 1740183762305810,
		"ts_ns": 1740183762305810000,
		"schema": "public",
		"table": "customers",
		"txId": 763,
		"lsn": 27606456,
		"xmin": null
	},
	"transaction": null,
	"op": "c",
	"ts_ms": 1740183762818,
	"ts_us": 1740183762818345,
	"ts_ns": 1740183762818345200
} and source stream 

In [71]:
data = """{
	"schema": {
		"type": "struct",
		"fields": [
			{
				"type": "string",
				"optional": true,
				"name": "io.debezium.data.Json",
				"version": 1,
				"field": "before"
			},
			{
				"type": "string",
				"optional": true,
				"name": "io.debezium.data.Json",
				"version": 1,
				"field": "after"
			},
			{
				"type": "struct",
				"fields": [
					{
						"type": "array",
						"items": {
							"type": "string",
							"optional": false
						},
						"optional": true,
						"field": "removedFields"
					},
					{
						"type": "string",
						"optional": true,
						"name": "io.debezium.data.Json",
						"version": 1,
						"field": "updatedFields"
					},
					{
						"type": "array",
						"items": {
							"type": "struct",
							"fields": [
								{
									"type": "string",
									"optional": false,
									"field": "field"
								},
								{
									"type": "int32",
									"optional": false,
									"field": "size"
								}
							],
							"optional": false,
							"name": "io.debezium.connector.mongodb.changestream.truncatedarray",
							"version": 1
						},
						"optional": true,
						"field": "truncatedArrays"
					}
				],
				"optional": true,
				"name": "io.debezium.connector.mongodb.changestream.updatedescription",
				"version": 1,
				"field": "updateDescription"
			},
			{
				"type": "struct",
				"fields": [
					{
						"type": "string",
						"optional": false,
						"field": "version"
					},
					{
						"type": "string",
						"optional": false,
						"field": "connector"
					},
					{
						"type": "string",
						"optional": false,
						"field": "name"
					},
					{
						"type": "int64",
						"optional": false,
						"field": "ts_ms"
					},
					{
						"type": "string",
						"optional": true,
						"name": "io.debezium.data.Enum",
						"version": 1,
						"parameters": {
							"allowed": "true,first,first_in_data_collection,last_in_data_collection,last,false,incremental"
						},
						"default": "false",
						"field": "snapshot"
					},
					{
						"type": "string",
						"optional": false,
						"field": "db"
					},
					{
						"type": "string",
						"optional": true,
						"field": "sequence"
					},
					{
						"type": "int64",
						"optional": true,
						"field": "ts_us"
					},
					{
						"type": "int64",
						"optional": true,
						"field": "ts_ns"
					},
					{
						"type": "string",
						"optional": false,
						"field": "collection"
					},
					{
						"type": "int32",
						"optional": false,
						"field": "ord"
					},
					{
						"type": "string",
						"optional": true,
						"field": "lsid"
					},
					{
						"type": "int64",
						"optional": true,
						"field": "txnNumber"
					},
					{
						"type": "int64",
						"optional": true,
						"field": "wallTime"
					}
				],
				"optional": false,
				"name": "io.debezium.connector.mongo.Source",
				"field": "source"
			},
			{
				"type": "string",
				"optional": true,
				"field": "op"
			},
			{
				"type": "int64",
				"optional": true,
				"field": "ts_ms"
			},
			{
				"type": "struct",
				"fields": [
					{
						"type": "string",
						"optional": false,
						"field": "id"
					},
					{
						"type": "int64",
						"optional": false,
						"field": "total_order"
					},
					{
						"type": "int64",
						"optional": false,
						"field": "data_collection_order"
					}
				],
				"optional": true,
				"name": "event.block",
				"version": 1,
				"field": "transaction"
			}
		],
		"optional": false,
		"name": "mongodb.lumi_data.unstructured_data.Envelope"
	},
	"payload": {
		"before": null,
		"after": "{\"_id\": {\"$oid\": \"67b918d21585c8c281a60f0e\"},\"customer_id\": 10,\"raw_data\": {\"transaction_history\": [{\"date\": {\"$date\": 1740009600000},\"amount\": 734.57},{\"date\": {\"$date\": 1739836800000},\"amount\": 631.99},{\"date\": {\"$date\": 1735862400000},\"amount\": 384.18},{\"date\": {\"$date\": 1739404800000},\"amount\": 921.92}],\"social_media_activity\": {\"platform\": \"LinkedIn\",\"activity_score\": 63},\"miscellaneous\": {\"notes\": \"Cell week per all power administration.\",\"risk_flags\": \"Low\"}}}",
		"updateDescription": null,
		"source": {
			"version": "3.0.6.Final",
			"connector": "mongodb",
			"name": "mongodb",
			"ts_ms": 1740183762000,
			"snapshot": "false",
			"db": "lumi_data",
			"sequence": null,
			"ts_us": 1740183762000000,
			"ts_ns": 1740183762000000000,
			"collection": "unstructured_data",
			"ord": 10,
			"lsid": null,
			"txnNumber": null,
			"wallTime": 1740183762308
		},
		"op": "c",
		"ts_ms": 1740183762403,
		"transaction": null
	}
}
}"""

source_stream_name = "kafka_cdc_mongo_unstructure"
target_stream_name = "mongo_unstructure"

agent = DataExtractionAgent()

agent1_output, agent2_output, agent3_output = agent.pipeline(data, source_stream_name, target_stream_name)

code = extract_code_blocks_with_type(agent1_output)
print(f"code type {code[0][0]}, extraction sql : {code[0][1]}")

print(agent2_output)

print(agent3_output)


********************************************************************************
Starting a new chat....

********************************************************************************
data_extraction_agent (to payload_extraction_agent):

based on input data : {
	"schema": {
		"type": "struct",
		"fields": [
			{
				"type": "string",
				"optional": true,
				"name": "io.debezium.data.Json",
				"version": 1,
				"field": "before"
			},
			{
				"type": "string",
				"optional": true,
				"name": "io.debezium.data.Json",
				"version": 1,
				"field": "after"
			},
			{
				"type": "struct",
				"fields": [
					{
						"type": "array",
						"items": {
							"type": "string",
							"optional": false
						},
						"optional": true,
						"field": "removedFields"
					},
					{
						"type": "string",
						"optional": true,
						"name": "io.debezium.data.Json",
						"version": 1,
						"field": "updatedFields"
					},
					{
						"type": "array",
						"items": {
							"type